In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
exporters = spark.sql('select * from workarea.us_export_propensity_exporters')

smad = spark.sql('select duns_nbr as duns, load_year as append_year, load_month as append_month from workarea.smad where load_year in (2016, 2017) and load_month = 9')

smad2 = smad.join(exporters, on = ['duns', 'append_year', 'append_month'], how = 'left')

smad2 = smad2.withColumn('export', when(col('export').isNull(), 0).otherwise(col('export')))

non_exporters = smad2.where('export = 0')

non_exporters_2016 = non_exporters.where('append_year = 2016')
non_exporters_2017 = non_exporters.where('append_year = 2017')

sample_non_exporters_2016 = non_exporters_2016.sample(0.001773034219845912)

sample_non_exporters_2017 = non_exporters_2017.sample(0.0013545224091702894)

sample_non_exporters = sample_non_exporters_2016.union(sample_non_exporters_2017)

analytic_dataset = exporters.union(sample_non_exporters)

analytic_dataset.write.mode('overwrite').saveAsTable('workarea.us_export_propensity_analytic_dataset')